Download all census datasets (5 year estimates from every year from 2010-2016). Format dataframes, drop columns with >40% NaN, and impute NaNs in the rest of the dataframe. Final output is a pkl of the dictionary where each enry is a dictionary for each of the datasets (economic, social, demographics, housing, and migration). Each dataset dictionary has an entry for each year.

Data source: downloaded on county level from the American Factfinder website.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as pl
import datetime
import pickle

from fancyimpute import KNN

In [2]:
def column_filter(df,phrase):
    '''produces a list of columns that contain the phrase'''
    start_len = len(df.columns)
    keep_cols = []
    for i in df.columns:
        if phrase in i:
            keep_cols.append(i)
    #print ('Columns matching filter: {}'.format(len(keep_cols)))
    return (keep_cols)

In [3]:
def drop_columns(df,phrase,dropped_cols=False):
    '''Drop all columns in dataframe df that contain substring phrase'''
    start_len = len(df.columns)
    drop_cols = []
    for i in df.columns:
        if phrase in i:
            drop_cols.append(i)
            df.drop([i], axis=1, inplace=True)
    #print ('Total Columns Dropped: {}'.format(len(drop_cols)))
    if dropped_cols == True:
        print ('Dropped Columns: {}'.format(drop_cols))

In [118]:
def process_acs_data(census_dict):
    '''Format census data and drop irrelevant columns'''
    for yr in census_dict.keys():
        census_dict[yr].columns = census_dict[yr].iloc[0]
        census_dict[yr] = census_dict[yr].reindex(census_dict[yr].index.drop(0))
        
        try:
            census_dict[yr].rename(columns={'Id2': 'FIPS'}, inplace=True) 
                                                  #if keeping descriptive features
            census_dict[yr].drop(['Id', 'Geography'], axis=1, inplace=True) 
                                                  #if keeping descriptive features
        except ValueError:
            pass

        #census_dict[yr].rename(columns={'GEO.id2': 'FIPS'}, inplace=True) 
                                                  #if keeping feature codes
        #census_dict[yr].drop(['index', 'GEO.id', 'GEO.display-label'], axis=1, inplace=True) 
                                                  #if keeping feature codes


        #dropping irrelevant features
        drop_columns(census_dict[yr], 'Margin of Error')
        perc = column_filter(census_dict[yr], 'Percent')
        mean = column_filter(census_dict[yr], 'Mean')
        median = column_filter(census_dict[yr], 'Median')
        a = (perc + mean  + median)
        a.append('FIPS')
        census_dict[yr] = census_dict[yr][a]


        #dropping values that are totals and not normalized
                        #-- assuming percents are there already
        try: 
            census_dict[yr].drop(['Percent; EMPLOYMENT STATUS - Civilian labor force',
             'Percent; EMPLOYMENT STATUS - Females 16 years and over',
             'Percent; EMPLOYMENT STATUS - Own children under 6 years',
             'Percent; EMPLOYMENT STATUS - Own children 6 to 17 years',
             'Percent; COMMUTING TO WORK - Workers 16 years and over',
             'Percent; OCCUPATION - Civilian employed population 16 years and over',
             'Percent; INDUSTRY - Civilian employed population 16 years and over',
             'Percent; CLASS OF WORKER - Civilian employed population 16 years and over',
             'Percent; INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS) - Total census_dicteholds',
             'Percent; INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS) - Families',
             'Percent; INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS) - Nonfamily census_dicteholds',
             'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population',
             'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population under 18 years',
             'Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population 18 to 64 years',
             'Percent; HEALTH INSURANCE COVERAGE - In labor force:',
             'Percent; HEALTH INSURANCE COVERAGE - In labor force: - Employed:',
             'Percent; HEALTH INSURANCE COVERAGE - In labor force: - Unemployed:',
             'Percent; HEALTH INSURANCE COVERAGE - Not in labor force:'], axis=1, inplace=True)
        except ValueError:
            pass

        #formatting FIPS and numeric
        census_dict[yr] = census_dict[yr].apply(pd.to_numeric, errors='coerce')
        census_dict[yr]['FIPS'] = census_dict[yr]['FIPS'].apply(str)
        census_dict[yr]['FIPS'] = census_dict[yr]['FIPS'].str.zfill(5)

        #dropping NaN columns
        census_dict[yr].dropna(axis=1, how='all', inplace=True)

        census_dict[yr].columns = census_dict[yr].columns.str.replace(' ', '_')
        census_dict[yr].columns = census_dict[yr].columns.str.replace(';', '_')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('.', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace(',', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('(', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace(')', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('"', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace("'", '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('-', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace(':', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('$', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2010', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2011', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2012', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2013', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2014', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2015', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('2016', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('__Civilian_employed_population_16_years_and_over', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('__Workers_16_years_and_over', '')
        
        try: 
            census_dict[yr].drop(['Percent__INCOME_AND_BENEFITS_IN__INFLATIONADJUSTED_DOLLARS__Families__150000_to_199999',
             'Percent__INCOME_AND_BENEFITS_IN__INFLATIONADJUSTED_DOLLARS__Families__150000_to_199999_log',
             'Percent__INCOME_AND_BENEFITS_IN__INFLATIONADJUSTED_DOLLARS__Families__200000_or_more',
             'Percent__INCOME_AND_BENEFITS_IN__INFLATIONADJUSTED_DOLLARS__Families__200000_or_more_log', 
             'Estimate__SELECTED_MONTHLY_OWNER_COSTS_SMOC__Housing_units_without_a_mortgage__Median_dollars',
             'Estimate__SELECTED_MONTHLY_OWNER_COSTS_SMOC__Housing_units_without_a_mortgage__Median_dollars_log'], axis=1, inplace=True)
        except ValueError:
            pass
        
        #census_dict[yr].columns = census_dict[yr].columns.str.replace('Families__', '')
        #census_dict[yr].columns = census_dict[yr].columns.str.replace('Nonfamily_households__', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Total_households__', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Occupied_units_paying_rent__', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Total_population__', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Housing_units_with_a_mortgage__', '') 
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Population_25_years_and_over__', '')

        census_dict[yr].columns = census_dict[yr].columns.str.replace('Owneroccupied_units__', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Total_housing_units__', '')
        #census_dict[yr].columns = census_dict[yr].columns.str.replace('Population_5_years_and_over__', '')
        census_dict[yr].columns = census_dict[yr].columns.str.replace('Race_alone_or_in_combination_with_one_or_more_other_races__', '')

        #dropping dup columns

        census_dict[yr] = census_dict[yr].loc[:,~census_dict[yr].columns.duplicated()]       
        print ('Columns after processing:{} \n'.format(len(census_dict[yr].columns)))                                         

    return census_dict

Load all dataframe dictionaries with county data, and combine multi years into single dataframes, indexed on FIPS  
- economic data --> ** econ_df **
    - FIPS, feature, time series by year (2010-2016)
    - FIPS is in str form with padded 0
    - feature codes are listed in this doc: ACS_14_1YR_DP03_metadata.csv 
    - be careful when merging later: demographic data (housing, econ, social, acs) all have the same codes that refer to different things. Add a prefix or suffix to denote which category the code refers to
- housing data --> ** hous_df  **
    - FIPS, feature, time series by year (2010-2016)
    - FIPS is str, padded 0
    - feature codes are listed in this doc: ACS_14_1YR_DP04_metadata.csv
- Social demographics --> ** soc_df **
    - FIPS, feature, time series by year (2010-2016)
    - FIPS is str, padded 0
    - feature codes are listed in this doc: ACS_14_1YR_DP02_metadata.csv
- Race, Age, gender--> ** dem_df **
    - FIPS, feature, time series by year (2010-2016)
    - FIPS is str, padded 0
    - feature codes are listed in this doc: ACS_14_1YR_DP05_metadata.csv
- population profile --> ** pop_df **
    - FIPS, feature, time series by year (2005-2016)
    - FIPS is str, padded 0
    - feature codes are listed in this doc: ACS_05_EST_S0201_metadata.csv
    - race codes are attached at the end of each feature code
        - for example: feature code EST_VC01 refers to the Estimate; Total population
        - the race code '001' refers to the total population
        - the race code '002' refers to whites
        - therefore, a feature with the code 'EST_VC01_001' would refer to the total population size in a particular county. A feature with the code 'EST_VC01-002' would refer to the total population size of only white constituents of that particular county.  
    - Race code references are stores in the dictionary: id_groups
- migration stats --> ** migr_df **
    - FIPS, feature, time series by year (2011-2016)
    - FIPS is str, padded 0
    - feature codes are listed in this doc: PEP_2011_PEPTCOMP_metadata.csv    

#### DEMOGRAPHIC DATA

## Loading Census DataFrames

** * Economic Data from ACS * **

In [142]:
year = np.arange(10,17)

In [143]:
econ = {}
for yr in year:
    path = './data/Demographic_Data/ACS_{}_1YR_DP03/ACS_{}_1YR_DP03_with_ann.csv'.format(yr, yr)
    econ[yr] =  pd.read_csv(path, encoding = "ISO-8859-1")
    print (yr, 'Columns: ', len(econ[yr].columns), 'Counties: ', len(econ[yr]))

10 Columns:  551 Counties:  808
11 Columns:  551 Counties:  812
12 Columns:  551 Counties:  815
13 Columns:  551 Counties:  818
14 Columns:  551 Counties:  818
15 Columns:  551 Counties:  820
16 Columns:  551 Counties:  821


In [144]:
econ = process_acs_data(econ)

Columns after processing:106 

Columns after processing:107 

Columns after processing:107 

Columns after processing:126 

Columns after processing:126 

Columns after processing:128 

Columns after processing:128 



** * Housing Data * **

In [145]:
hous = {}
for yr in year:
    path = './data/Demographic_Data/ACS_{}_1YR_DP04/ACS_{}_1YR_DP04_with_ann.csv'.format(yr, yr)
    hous[yr] =  pd.read_csv(path, encoding = "ISO-8859-1")
    print (yr, 'Columns: ', len(hous[yr].columns), 'Counties: ', len(hous[yr]))

10 Columns:  567 Counties:  808
11 Columns:  567 Counties:  812
12 Columns:  567 Counties:  815
13 Columns:  567 Counties:  818
14 Columns:  567 Counties:  818
15 Columns:  575 Counties:  820
16 Columns:  575 Counties:  821


In [146]:
hous = process_acs_data(hous)

Columns after processing:129 

Columns after processing:129 

Columns after processing:129 

Columns after processing:134 

Columns after processing:134 

Columns after processing:136 

Columns after processing:136 



** * Social Demographics Data * **

In [147]:
soc = {}
for yr in year:
    path = './data/Demographic_Data/ACS_{}_1YR_DP02/ACS_{}_1YR_DP02_with_ann.csv'.format(yr, yr)
    soc[yr] =  pd.read_csv(path, encoding = "ISO-8859-1")
    print (yr, 'Columns: ', len(soc[yr].columns), 'Counties: ', len(soc[yr]))

10 Columns:  599 Counties:  808
11 Columns:  599 Counties:  812
12 Columns:  599 Counties:  815
13 Columns:  611 Counties:  818
14 Columns:  611 Counties:  818
15 Columns:  611 Counties:  820
16 Columns:  611 Counties:  821


In [148]:
soc = process_acs_data(soc)

Columns after processing:133 

Columns after processing:133 

Columns after processing:133 

Columns after processing:146 

Columns after processing:146 

Columns after processing:146 

Columns after processing:146 



** * Race, Age, Gender * **

In [149]:
dem = {}
for yr in year:
    path = './data/Demographic_Data/ACS_{}_1YR_DP05/ACS_{}_1YR_DP05_with_ann.csv'.format(yr, yr)
    dem[yr] =  pd.read_csv(path, encoding = "ISO-8859-1")
    print (yr, 'Columns: ', len(dem[yr].columns), 'Counties: ', len(dem[yr]))

10 Columns:  327 Counties:  808
11 Columns:  327 Counties:  812
12 Columns:  327 Counties:  815
13 Columns:  327 Counties:  818
14 Columns:  327 Counties:  818
15 Columns:  339 Counties:  820
16 Columns:  339 Counties:  821


In [150]:
dem = process_acs_data(dem)

Columns after processing:72 

Columns after processing:72 

Columns after processing:72 

Columns after processing:76 

Columns after processing:76 

Columns after processing:79 

Columns after processing:79 



** * birth, death, migrations * **

In [151]:
years = np.arange(2011, 2017)

In [152]:
migr = {}
for yr in years:
    path = './data/County_Snapshots/Births_Death_Migration/PEP_{}_PEPTCOMP/PEP_{}_PEPTCOMP_with_ann.csv'.format(yr, yr)
    migr[yr] =  pd.read_csv(path, encoding = "ISO-8859-1")

    #replace the features codes with descriptive features
    migr[yr].columns = migr[yr].iloc[0]
    migr[yr] = migr[yr].reindex(migr[yr].index.drop(0))
    migr[yr]['Year'] = str(yr)
    
    try:
        migr[yr].rename(columns={'Id2': 'FIPS'}, inplace=True) #if keeping descriptive features
        migr[yr].drop(['Id', 'Geography'], axis=1, inplace=True) #if keeping descriptive features
    except ValueError:
        pass
    
    migr[yr].columns = migr[yr].columns.str.replace('2010', '')
    migr[yr].columns = migr[yr].columns.str.replace('2011', '')
    migr[yr].columns = migr[yr].columns.str.replace('2012', '')
    migr[yr].columns = migr[yr].columns.str.replace('2013', '')
    migr[yr].columns = migr[yr].columns.str.replace('2014', '')
    migr[yr].columns = migr[yr].columns.str.replace('2015', '')
    migr[yr].columns = migr[yr].columns.str.replace('2016', '')
    
    migr[yr].iloc[:, 1:] = migr[yr].iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    
    print (yr, 'Columns: ', len(migr[yr].columns), 'Counties: ', len(migr[yr]))

2011 Columns:  16 Counties:  3143
2012 Columns:  16 Counties:  3143
2013 Columns:  16 Counties:  3143
2014 Columns:  16 Counties:  3142
2015 Columns:  16 Counties:  3142
2016 Columns:  16 Counties:  3142


There are way more counties in the migration dataset than in the other census datasets.  

## Handling NaNs

In [153]:
comm_dicts = [econ, soc, migr, dem, hous]

In [154]:
#setting dictionaries to have the same keys
for d in [econ, soc, dem, hous]:
    for yr in year:
        new_key = yr + 2000
        d[new_key] = d.pop(yr)

In [155]:
for d in comm_dicts:
    for yr in d.keys():
        print (yr, d[yr].isnull().sum().sort_values(ascending=False)[:10], '\n')

2010 0
Estimate__INCOME_AND_BENEFITS_IN__INFLATIONADJUSTED_DOLLARS__With_cash_public_assistance_income__Mean_cash_public_assistance_income_dollars    53
Percent__OCCUPATION__Natural_resources_construction_and_maintenance_occupations                                                                 7
Percent__OCCUPATION__Production_transportation_and_material_moving_occupations                                                                  7
Percent__OCCUPATION__Sales_and_office_occupations                                                                                               7
Percent__OCCUPATION__Service_occupations                                                                                                        7
Percent__OCCUPATION__Management_business_science_and_arts_occupations                                                                           7
Percent__COMMUTING_TO_WORK__Car_truck_or_van__drove_alone                                                            

The migration dataset has missing many values because it has so many more counites than the demographic data that was used to calculate percents. For the census data, it makes sense to drop the columns that have more than 40% of the data missing, and to impute the missing values for the columns that are less than 40% NaN. 

In [156]:
for d in [econ, soc, dem, hous]:
    for yr in d.keys():
        drp_col = []
        for i in d[yr].columns:
            if d[yr][i].isnull().sum() / len(d[yr]) > .4:
                drp_col.append(i)
        d[yr].drop(drp_col, axis=1, inplace=True)
        print (yr, 'columns dropped: ', len(drp_col))
        #d[yr] = KNN(k=3).complete(d[yr])
        #d[yr] = pd.DataFrame(d[yr])

2010 columns dropped:  0
2011 columns dropped:  0
2012 columns dropped:  0
2013 columns dropped:  0
2014 columns dropped:  0
2015 columns dropped:  0
2016 columns dropped:  0
2010 columns dropped:  26
2011 columns dropped:  26
2012 columns dropped:  19
2013 columns dropped:  26
2014 columns dropped:  22
2015 columns dropped:  22
2016 columns dropped:  22
2010 columns dropped:  20
2011 columns dropped:  20
2012 columns dropped:  20
2013 columns dropped:  20
2014 columns dropped:  20
2015 columns dropped:  16
2016 columns dropped:  20
2010 columns dropped:  0
2011 columns dropped:  0
2012 columns dropped:  0
2013 columns dropped:  0
2014 columns dropped:  0
2015 columns dropped:  0
2016 columns dropped:  0


In [157]:
for d in [econ, soc, dem, hous]:
    for yr in d.keys():
        col_names = d[yr].columns
        print (yr, d[yr].isnull().sum().sum())
        d[yr] = KNN(k=3).complete(d[yr])
        d[yr] = pd.DataFrame(d[yr])
        d[yr].columns = col_names

2010 135
Imputing row 1/807 with 0 missing, elapsed time: 0.346
Imputing row 101/807 with 0 missing, elapsed time: 0.349
Imputing row 201/807 with 0 missing, elapsed time: 0.349
Imputing row 301/807 with 0 missing, elapsed time: 0.350
Imputing row 401/807 with 0 missing, elapsed time: 0.351
Imputing row 501/807 with 0 missing, elapsed time: 0.351
Imputing row 601/807 with 0 missing, elapsed time: 0.352
Imputing row 701/807 with 0 missing, elapsed time: 0.353
Imputing row 801/807 with 0 missing, elapsed time: 0.354
2011 410
Imputing row 1/811 with 0 missing, elapsed time: 0.338
Imputing row 101/811 with 0 missing, elapsed time: 0.339
Imputing row 201/811 with 0 missing, elapsed time: 0.342
Imputing row 301/811 with 0 missing, elapsed time: 0.343
Imputing row 401/811 with 0 missing, elapsed time: 0.345
Imputing row 501/811 with 0 missing, elapsed time: 0.346
Imputing row 601/811 with 0 missing, elapsed time: 0.346
Imputing row 701/811 with 6 missing, elapsed time: 0.349
Imputing row 801/

Imputing row 1/814 with 0 missing, elapsed time: 0.294
Imputing row 101/814 with 0 missing, elapsed time: 0.297
Imputing row 201/814 with 0 missing, elapsed time: 0.303
Imputing row 301/814 with 0 missing, elapsed time: 0.311
Imputing row 401/814 with 0 missing, elapsed time: 0.316
Imputing row 501/814 with 4 missing, elapsed time: 0.318
Imputing row 601/814 with 0 missing, elapsed time: 0.327
Imputing row 701/814 with 0 missing, elapsed time: 0.333
Imputing row 801/814 with 0 missing, elapsed time: 0.339
2013 2094
Imputing row 1/817 with 4 missing, elapsed time: 0.339
Imputing row 101/817 with 0 missing, elapsed time: 0.344
Imputing row 201/817 with 0 missing, elapsed time: 0.349
Imputing row 301/817 with 0 missing, elapsed time: 0.361
Imputing row 401/817 with 0 missing, elapsed time: 0.368
Imputing row 501/817 with 4 missing, elapsed time: 0.373
Imputing row 601/817 with 0 missing, elapsed time: 0.379
Imputing row 701/817 with 14 missing, elapsed time: 0.389
Imputing row 801/817 wit

In [158]:
for d in comm_dicts:
    for yr in d.keys():
        print (yr, 'Columns: ', len(d[yr].columns), 'Rows: ', len(d[yr]), 
               'NaNs: ', d[yr].isnull().sum().sum())


2010 Columns:  106 Rows:  807 NaNs:  0
2011 Columns:  107 Rows:  811 NaNs:  0
2012 Columns:  107 Rows:  814 NaNs:  0
2013 Columns:  126 Rows:  817 NaNs:  0
2014 Columns:  126 Rows:  817 NaNs:  0
2015 Columns:  128 Rows:  819 NaNs:  0
2016 Columns:  128 Rows:  820 NaNs:  0
2010 Columns:  107 Rows:  807 NaNs:  0
2011 Columns:  107 Rows:  811 NaNs:  0
2012 Columns:  114 Rows:  814 NaNs:  0
2013 Columns:  120 Rows:  817 NaNs:  0
2014 Columns:  124 Rows:  817 NaNs:  0
2015 Columns:  124 Rows:  819 NaNs:  0
2016 Columns:  124 Rows:  820 NaNs:  0
2011 Columns:  16 Rows:  3143 NaNs:  0
2012 Columns:  16 Rows:  3143 NaNs:  0
2013 Columns:  16 Rows:  3143 NaNs:  0
2014 Columns:  16 Rows:  3142 NaNs:  0
2015 Columns:  16 Rows:  3142 NaNs:  0
2016 Columns:  16 Rows:  3142 NaNs:  0
2010 Columns:  52 Rows:  807 NaNs:  0
2011 Columns:  52 Rows:  811 NaNs:  0
2012 Columns:  52 Rows:  814 NaNs:  0
2013 Columns:  56 Rows:  817 NaNs:  0
2014 Columns:  56 Rows:  817 NaNs:  0
2015 Columns:  63 Rows:  819 N

## Saving the output

In [159]:
com_dict_keys = ['econ', 'soc', 'migr', 'dem', 'hous']

In [160]:
census_data = {}
for i in np.arange(len(comm_dicts)):
    census_data[com_dict_keys[i]] = comm_dicts[i]

In [161]:
output = open('./data/dataframes/CensusData.pkl', 'wb')
pickle.dump(census_data, output)
output.close()